In [3]:
import pandas as pd
import numpy as np
import pickle
import gzip
from multiprocessing import Pool
from sklearn.model_selection import train_test_split
import yaml
from xgboost import XGBClassifier
with open("configuration.yaml", "r") as yml_file:
    config = yaml.load(yml_file, yaml.Loader)

COLUMNS_TRAINING = config['COLUMNS_TRAINING']

In [41]:
def sort_filter(x_train, x_test, y_train, y_test):
    """
    Final preparation steps to achieve a competent CV data:
    1) remove repeated data items in the test datasets
       removing duplicate sites in test dataset --but not in training-- as repeated data in training provide us with
       weight of evidence, whereas too many repeated data at testing can spoil our capacity to evaluate
    2) random sampling to get a balanced test set
    3) set training feature labels in a canonical order taken from configuration
    """

    # reset index

    x_test.reset_index(inplace=True, drop=True)
    y_test.reset_index(inplace=True, drop=True)

    # remove duplicates from test set

    x_test['chr'] = x_test['chr'].astype(str)
    x_test['start'] = x_test['start'].astype(int)
    x_test = x_test.drop_duplicates(['start', 'alt'])
    test_index = x_test.index
    y_test = y_test.loc[y_test.index.intersection(test_index)]

    # balance test set

    total_index = set(x_test.index)
    if y_test.mean() <= 0.5:
        balance_index = y_test[y_test == 1].index.tolist()
    else:
        balance_index = y_test[y_test == 0].index.tolist()
    remaining_index = list(set(total_index) - set(balance_index))
    balance_index += list(np.random.choice(remaining_index, size=len(balance_index), replace=False))
    x_test = x_test.loc[x_test.index.intersection(balance_index)]
    y_test = y_test.loc[y_test.index.intersection(balance_index)]

    # feature labels in standard order
    avoid = ['chr', 'start', 'ref', 'alt']
    features = list(filter(lambda x: x not in avoid, x_train))
    # print(set(features))
    # assert (set(features) == set(COLUMNS_TRAINING))
    x_train = x_train[avoid + COLUMNS_TRAINING]
    x_test = x_test[avoid + COLUMNS_TRAINING]

    return x_train, x_test, y_train, y_test

def split_balanced(x_data, y_data, test_size=0.3):
    """Generate balanced train-test split"""

    one_index  = list(y_data[y_data == 1].index)
    zero_index = list(y_data[y_data == 0].index)

    # randomly select n_ones indices from zero_index
    zero_index = list(np.random.choice(zero_index, size=len(one_index), replace=False))

    x_data_sub = x_data.loc[one_index + zero_index, :]
    y_data_sub = y_data.loc[one_index + zero_index]

    # the random state should be fixed prior to this call
    x_train, x_test, y_train, y_test = train_test_split(x_data_sub, y_data_sub, test_size=test_size)
    return x_train, x_test, y_train, y_test

def get_cv_sets_balanced(x_data, y_data, n, size):
    """Generate several balanced train-test sets"""

    for _ in range(n):
        x_train, x_test, y_train, y_test = split_balanced(x_data, y_data, test_size=size)
        yield x_train, x_test, y_train, y_test

def prepare(data, nsplits=10, test_size=0.2):

    data.rename(columns={'response': 'label'}, inplace=True)

    # keep 'pos' and 'chr' to run position-based filtering
    avoid = ['cohort', 'gene', 'aachange', 'label', 'motif']  # include 'ref' 'alt'
    features = list(filter(lambda x: x not in avoid, data.columns))

    # regression datasets
    x_data = data[features]
    y_data = data['driver'] # label

    # cv_list output has tuples (x_train, x_test, y_train, y_test) as elements
    cv_list = get_cv_sets_balanced(x_data, y_data, nsplits, test_size)

    # filter test data to prevent site repetitions
    cv_list = [sort_filter(*arg) for arg in cv_list]

    return cv_list

In [37]:
def load_mutations(input_path):

    data = pd.read_csv(input_path, sep=',', low_memory=False)
    return data

def generate(mutations, random_state=None, bootstrap_splits=50, cv_fraction=0.3):

    # fix random seed (if provided)
    np.random.seed(random_state)

    d_output = {}

    # for gene, data in mutations.groupby('gene'):
    cv_list = prepare(mutations.copy(), nsplits=bootstrap_splits, test_size=cv_fraction) # data.copy()
    d_output = cv_list

    return d_output

In [42]:
mutations = load_mutations('data/final_dataset.csv')

mutations.drop(['chr_y', 'start_y', 'end_y'], inplace=True, axis=1)
mutations.rename({'chr_x': 'chr', 'start_x': 'start', 'end_x': 'end'}, inplace=True, axis=1)

d_output = generate(mutations,
                    random_state=104,
                    bootstrap_splits=50,
                    cv_fraction=0.3)

with gzip.open('data/splits.pkl', 'wb') as f:
    pickle.dump(d_output, f)

In [46]:
len(mutations)

3102

# Training

In [ ]:
dict_results = {
        'models': [], 'split_number': [], 'x_test': [], 'y_test': [], 'learning_curves': []
    }

In [ ]:
def train(values):
    """Returns: optimal classification threshold and trained XGB model"""

    x_train, x_test, y_train, y_test, split_number, seed = tuple(values)
    XGB_PARAMS = config['XGB_PARAMS']
    params = XGB_PARAMS.copy()
    params['n_estimators'] = 20000  # set it high enough to allow "early stopping" events below
    params['base_score'] = y_train.mean()
    params['n_jobs'] = 1
    params['seed'] = seed
    myclassifier = XGBClassifier(**params)

    # train with xgboost
    learning_curve_dict = {}
    myclassifier.train(x_train, y_train,
                       eval_set=[(x_train, y_train), (x_test, y_test)],
                       eval_metric='logloss',  # mcc_loss could be used here
                       early_stopping_rounds=2000,
                       callbacks=[
                           xgboost.callback.record_evaluation(learning_curve_dict),
                       ],
                       verbose=False)

    params['n_estimators'] = myclassifier.model.best_iteration
    learning_curve_dict = {k: v['logloss'][:params['n_estimators']] for k, v in learning_curve_dict.items()}
    myclassifier.model.set_params(**params)

    return myclassifier, split_number, x_test, y_test, learning_curve_dict

file_cv = 'data/splits.pkl'
min_rows = 30   # Minimum number of rows to carry out training
non_features = ['pos', 'chr', 'ref', 'alt']
cores = 1
with Pool(cores) as p:
    with gzip.open(file_cv, 'rb') as f:
        split_cv = pickle.load(f)

    mean_size = np.nanmean([cv[0].shape[0] for cv in split_cv])
    if mean_size < min_rows:
        print("ERROR")

    list_cvs = []
    for i, x in enumerate(split_cv):
        x_list = list(x) + [i, np.random.randint(100000)]

        # filter out non-features, i.e., columns not used for training
        x_list[0] = x_list[0].drop(non_features, axis=1)
        x_list[1] = x_list[1].drop(non_features, axis=1)
        list_cvs.append(x_list)

    for model, split_number, x_test, y_test, learning_curve in p.imap(
            train, list_cvs):
        dict_results['models'].append(model)
        dict_results['split_number'].append(split_number)
        dict_results['x_test'].append(x_test)
        dict_results['y_test'].append(y_test)
        dict_results['learning_curves'].append(learning_curve)

    with gzip.open(output_file, 'wb') as f:
    pickle.dump(dict_results, f)